In [ ]:
import pandas as pd
df=pd.read_csv('7500covid.csv')


In [6]:
from nltk.corpus import stopwords, words
from nltk.tokenize import word_tokenize
import nltk
import re
import string
import pandas as pd
import numpy as np
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt_tab')
# Initialize stop words, English words corpus, and stemmer
stop_words = set(stopwords.words('english'))
english_words = set(words.words())
stop_words = set(stopwords.words("english"))

def preprocess(text):
    text = text.lower()

    text = re.sub(r"@\w+", "", text)

    text = re.sub(r"#(\w+)", r"\1", text)

    text = re.sub(r"http\S+|www\S+", "", text)

    text = text.translate(str.maketrans("", "", string.punctuation))

    text = ''.join([i for i in text if not i.isdigit()])

    tokens = word_tokenize(text)

    tokens = [word for word in tokens if word not in stop_words]

    return ' '.join(tokens)

df['Cleantext'] = df['original_text'].apply(preprocess)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\samgh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\samgh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\samgh\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\samgh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [7]:
from textblob import TextBlob
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np
import pandas as pd


label_mapping_Y = {'neg': 'negative', 'neu': 'neutral', 'pos': 'positive'}
Y_std = df['sentiment'].replace(label_mapping_Y)
X = df['Cleantext']


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, Y_std, test_size=1500, random_state=42, stratify=Y_std
)


from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(Y_std)

y_test_encoded = label_encoder.transform(y_test)


def textblob_to_class(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.05:
        return 'positive'
    elif polarity < -0.05:
        return 'negative'
    else:
        return 'neutral'

y_test_pred = X_test.apply(textblob_to_class)
y_test_pred_num = label_encoder.transform(y_test_pred)


def evaluate_metrics(y_true, y_pred):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted', zero_division=0),
        'recall': recall_score(y_true, y_pred, average='weighted', zero_division=0),
        'f1_score': f1_score(y_true, y_pred, average='weighted', zero_division=0)
    }

test_metrics = evaluate_metrics(y_test_encoded, y_test_pred_num)


class_names = label_encoder.classes_
print("Test classification report (TextBlob):")
print(classification_report(y_test_encoded, y_test_pred_num, target_names=class_names, zero_division=0))

print("Test metrics (TextBlob):", test_metrics)


Test classification report (TextBlob):
              precision    recall  f1-score   support

    negative       0.57      0.27      0.37       500
     neutral       0.39      0.59      0.47       500
    positive       0.46      0.47      0.47       500

    accuracy                           0.45      1500
   macro avg       0.47      0.45      0.44      1500
weighted avg       0.47      0.45      0.44      1500

Test metrics (TextBlob): {'accuracy': 0.44533333333333336, 'precision': 0.4747738740441339, 'recall': 0.44533333333333336, 'f1_score': 0.4362627332865838}
